In [5]:
import pandas as pd
import os
import fnmatch
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import scipy
import scipy.ndimage

def get_subdirectories_with_measurements(directory: str, pattern: str):
    matches = []
    for root, dirs, files in os.walk(directory):
        for subdir in dirs:
            if fnmatch.fnmatch(subdir, pattern):
                matches.append(os.path.join(root, subdir))
    return matches

def get_timed_measurements(path: str) -> pd.DataFrame:
    measurements = pd.read_csv(f"{path}/measurements.csv", decimal='.')

    timings = measurements.tail(2)
    timings.columns = ['timestamp']

    start = timings['timestamp'].min()
    end = timings['timestamp'].max()
    runtime = end - start

    # Omit the last two rows
    measurements = measurements.iloc[:-2]
    measurements.columns = ['current']

    time_between_measurements = runtime / measurements.shape[0]

    measurements['time'] = measurements.apply(lambda row: time_between_measurements * row.name, axis=1)
    measurements['timestamp'] = measurements.apply(lambda row: start + time_between_measurements * row.name, axis=1)
    smoothed = scipy.ndimage.gaussian_filter(measurements['current'], sigma=2)
    measurements['current_smoothed'] = smoothed

    return measurements

def get_timed_events(path: str, start: int) -> pd.DataFrame:
    event_columns = ['timestamp', 'label', 'time']
    events = pd.read_csv(f"{path}/events.csv", header=None, names=event_columns,) if os.path.exists(f"{path}/events.csv") else pd.DataFrame(columns=event_columns)
    if (not events.empty):    
        # timestamps_for_this_run = events # [(fmnist_timestamps['timestamp'] > start) & (fmnist_timestamps['timestamp'] < end)]
        events['time'] = events['timestamp'].apply(lambda timestamp: timestamp - start)
    return events

def create_graphs_for_experiment(path):
    measurements = get_timed_measurements(path)

    start = measurements['timestamp'].min()

    fig = px.scatter(measurements, x='time', y='current')
    fig.add_trace(go.Scatter(x=measurements['time'], y=measurements['current_smoothed']))

    events = get_timed_events(path, start)
    for index, row in events.iterrows():
        fig.add_vline(x=row['time'], line_width=3, line_color="green", annotation=dict(text=row['label'], textangle=-90))

    fig.add_hline(y=55*1000, line_dash="dot",
                annotation_text="baseline", 
                annotation_position="bottom right")

    filepath = f"{path}/plot.png"
    print(f"Saving to {filepath}")
    pio.write_image(fig, file=filepath, format='png', width=1920, height=1080)

for path in get_subdirectories_with_measurements('.', 'measurements_*'):
    print(f'Plotting {path}')
    create_graphs_for_experiment(path)

Plotting ./measurements_roberta_partial_0527145946
Saving to ./measurements_roberta_partial_0527145946/plot.png


KeyboardInterrupt: 

In [5]:
# Lets see how they overlap

experiments = ['sleep', 'roberta_stop_saved', 'roberta_stop_epoch', 'roberta_partial', 'roberta_full', 'imports_only']

def plot_multiple_experiments(name: str):
    folder_paths = get_subdirectories_with_measurements('.', f'measurements_{name}')
    all_experiments = pd.DataFrame()
    all_events = pd.DataFrame(columns=['timestamp', 'label', 'time', 'id'], data=[])

    fig = go.Figure()


    for index, path in enumerate(folder_paths):
        measurements = get_timed_measurements(path)
        measurements['id'] = index # give each experiment a different id, so we can differentiate them later

        fig.add_trace(go.Scatter(x=measurements['time'], y=measurements['current_smoothed']))

        start = measurements['timestamp'].min()
        events = get_timed_events(path, start)
        events['id'] = index

        all_events = pd.concat([all_events, events], ignore_index=True)

        #fig.add_trace(go.Scatter(x=df['time'], y=df['current']))

    fig.update_layout({'title': experiment, 'yaxis_title': 'Current in TODO unit', 'xaxis_title': 'Time since start in seconds'})

    grouped_events = all_events.groupby('label')
    for category, group in grouped_events:
        group = group.reset_index(drop=True)
        std = group['time'].std(skipna=True)
        average_time = pd.to_numeric(group['time']).mean()
        fig.add_vline(x=average_time, line_width=2, line_color="green", annotation=dict(text=f"{category}, std={std}", textangle=-90))

    fig.show()
    
    pass

for experiment in experiments:
    plot_multiple_experiments(experiment)

/tmp/ipykernel_62007/984792488.py:28: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

